In [ ]:
import os
import random
import time
from dataclasses import dataclass

import gymnasium as gym
import torch.nn.functional as F
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tyro
from torch.distributions.categorical import Categorical
#from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import copy

In [ ]:
@dataclass
class Args:
    exp_name: str = 'ppo'
    """the name of this experiment"""
    seed: int = 12
    """seed of the experiment"""
    torch_deterministic: bool = True
    """if toggled, `torch.backends.cudnn.deterministic=False`"""
    cuda: bool = True
    """if toggled, cuda will be enabled by default"""
    track: bool = False
    """if toggled, this experiment will be tracked with Weights and Biases"""
    capture_video: bool = False
    """whether to capture videos of the agent performances (check out `videos` folder)"""

    # Algorithm simulation arguments
    env_id: str = "CartPole-v1"
    """the id of the environment"""
    total_timesteps: int = 1000
    """total timesteps of the experiments"""
    num_envs: int = 4
    """the number of parallel game environments"""
    num_steps: int = 128
    """the number of steps to run in each environment per policy rollout"""
    gamma: float = 0.99
    """the discount factor gamma"""

    #Frequency simulation arguments
    M: int = 1000
    """ the number of frequency samples is M"""
    sigmaOmega: float = 1
    """ the standard deviation of the normal distribution from which the frequency is sampled """
    
    # traing arguments for critic
    batch_size: int = 2048
    """batch_size for critic training"""
    epochs: int = 4
    """number of epochs for critic training"""
    learning_rate: float = 2.5e-4
    """learning for critic training"""
    lw: float = 0.0
    """L2 regularization coefficient for critic training"""
    
    #to be filled at runtime
    num_iterations: int = 0
    """the number of iterations (computed in runtime)"""
    data_size_critic: int = 0
    """the number of dataset points over which critic is trained on (per iteration)"""
    critic_state_size: int = 0
    """the number of states over which critic is trained on (per iteration)"""

In [ ]:
args = Args()
args.critic_state_size = int(args.num_envs * args.num_steps)
args.num_iterations = args.total_timesteps // args.critic_state_size

run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{int(time.time())}"
#writer = SummaryWriter(f"runs/{run_name}")
#writer.add_text(
#        "hyperparameters",
#        "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(args).items()])),
#    )
print(args.num_iterations)

In [ ]:
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.backends.cudnn.deterministic = args.torch_deterministic
device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")

In [ ]:
def make_env(env_id, idx, capture_video, run_name):
    def thunk():
        if capture_video and idx == 0:
            env = gym.make(env_id, render_mode="rgb_array")
            env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        else:
            env = gym.make(env_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        return env

    return thunk

In [ ]:
def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    
    return layer

class xsinx(nn.Module):
    def __init__(self, a=1.0):
        super(xsinx, self).__init__()
        self.a = nn.Parameter(torch.tensor(a))

    def forward(self, x):
        return x + (1.0 / self.a) * torch.sin(self.a * x) ** 2

In [ ]:
class Agent(nn.Module):
    def __init__(self, envs):
        super().__init__()
        self.actor = nn.Sequential(
            layer_init(nn.Linear(np.array(envs.single_observation_space.shape).prod(), 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, 64)),
            nn.Tanh(),
            layer_init(nn.Linear(64, envs.single_action_space.n), std=0.01),
        )

    def get_action(self, x, action=None):
        logits = self.actor(x)
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action

In [ ]:
class cf(torch.nn.Module):
    def __init__(self, state_input_size, layer_sizes, sigma, sigma_opt=False):
        super().__init__()
        # Output size is twice the complex output size to account for real and imaginary parts
        output_size = 2
        self.layer_sizes = [state_input_size + 1] + layer_sizes + [output_size]
        self.layers = torch.nn.ModuleList()

        for i in range(len(self.layer_sizes) - 2):
            self.layers.append(layer_init(torch.nn.Linear(self.layer_sizes[i], self.layer_sizes[i + 1])))
            self.layers.append(xsinx(a=0.5))
        
        self.layers.append(layer_init(torch.nn.Linear(self.layer_sizes[-2], self.layer_sizes[-1])))

        if sigma_opt:
            self.sigma = torch.nn.Parameter(sigma)
        else:
            self.sigma = sigma

    def freq_sampling(self, num_samples):

        std_sample = torch.distributions.Normal(0.0, 1.0).sample((num_samples,)).to(device)
        return self.sigma * std_sample

    def forward(self,x):

        if x.dim() == 1:
            x = x.unsqueeze(0)

        for i, layer in enumerate(self.layers):
            x = layer(x)

        out = torch.zeros_like(x)
        out[:, 0] = torch.cos(x[:, 0])
        out[:, 1] = torch.sin(x[:, 1])        

        # Split the output into real and imaginary parts and form the complex output
        real = out[:,0]
        imag = out[:,1]
        return torch.complex(real, imag)

    def get_mean(self,state):
        omega_fixed = torch.tensor([0.0], requires_grad=True).expand(state.size(0)).unsqueeze(1).to(device)
        eva = torch.cat((state,omega_fixed),dim=1)
        output = self.forward(eva).imag
        gradients = torch.autograd.grad(outputs=output, inputs=eva, grad_outputs=torch.ones_like(output),allow_unused=True)
        return gradients[0][:,-1]

In [ ]:
cf_network = cf(4, [64, 64,64], 0.1,sigma_opt=False).to(device)
model_optimizer = optim.Adam(cf_network.parameters(), weight_decay = args.lw)

In [ ]:
envs = gym.vector.SyncVectorEnv(
        [make_env(args.env_id, i, args.capture_video, run_name) for i in range(args.num_envs)],
    )
agent = Agent(envs).to(device)

In [ ]:
obs = torch.zeros((args.num_steps, args.num_envs) + envs.single_observation_space.shape).to(device)
actions = torch.zeros((args.num_steps, args.num_envs) + envs.single_action_space.shape).to(device)
rewards = torch.zeros((args.num_steps, args.num_envs)).to(device)
dones = torch.zeros((args.num_steps, args.num_envs)).to(device)

input_data = torch.zeros((args.num_steps, args.num_envs*args.M)+envs.single_observation_space.shape).to(device)
output_data = torch.zeros((args.num_steps,args.num_envs*args.M),dtype=torch.complex128)

In [ ]:
global_step = 0
start_time = time.time()
next_obs, _ = envs.reset(seed=args.seed)
next_obs = torch.Tensor(next_obs).to(device)
next_done = torch.zeros(args.num_envs).to(device) 

In [ ]:
def td_update(r, t, ns, model,ndone):
    t_expanded = t.unsqueeze(0).expand(r.size(0), -1)
    input_tensor = torch.cat((ns, args.gamma*t_expanded), dim=-1)
    with torch.no_grad():
        return (1.0-ndone)*torch.exp(1j*t*r).flatten() + ndone*model(input_tensor).flatten()*torch.exp(1j*t*r).flatten()

# Function to apply td_update for a single t value
def single_td(r, ns, t, model,ndone):
    return td_update(r, t, ns, model,ndone)

# Vectorize over t values using vmap
def vectorized_td(r, ns, t_list, model,ndone):
    return torch.vmap(lambda t: single_td(r, ns, t, model,ndone), in_dims=0, out_dims=0)(t_list)

In [ ]:
model_copies = []
for iteration in range(1, args.num_iterations + 1):

    std_sample = torch.randn(args.M).to(device)
    omega = (args.sigmaOmega * std_sample).to(device)
    #omega = torch.cat((omega, (torch.zeros(3*args.M)).to(device)), dim=0).to(device)
    
    for step in range(0, args.num_steps):
        
        global_step += args.num_envs
        obs[step] = next_obs
        dones[step] = next_done

        with torch.no_grad():
            action = agent.get_action(next_obs)
        actions[step] = action
    
        next_obs, reward, terminations, truncations, infos = envs.step(action.cpu().numpy())
        next_done = np.logical_or(terminations, truncations)
        rewards[step] = 0.25*torch.tensor(reward).to(device).view(-1)
        next_obs, next_done = torch.Tensor(next_obs).to(device), torch.Tensor(next_done).to(device)

        if "final_info" in infos:
            for info in infos["final_info"]:
                if info and "episode" in info:
                    print(f"global_step={global_step}, episodic_return={info['episode']['r']}")
                    #writer.add_scalar("charts/episodic_return", info["episode"]["r"], global_step)
                    #writer.add_scalar("charts/episodic_length", info["episode"]["l"], global_step)

    with torch.no_grad():
        returns = torch.zeros_like(rewards).to(device)
        last = 0
        for t in reversed(range(args.num_steps)):
            if t == args.num_steps - 1:
                nextnonterminal = 1.0 - next_done
                output_data[t] = vectorized_td(rewards[t], next_obs, omega, cf_network,nextnonterminal).transpose(0,1).flatten()
            else:
                nextnonterminal = 1.0 - dones[t + 1]
                output_data[t] = vectorized_td(rewards[t], obs[t+1], omega, cf_network,nextnonterminal).transpose(0,1).flatten()

            delta = rewards[t]
            returns[t] = last = delta + args.gamma*nextnonterminal * last
            input_data[t] = obs[t].repeat_interleave(omega.shape[0], dim=0).to(device)
            
        
        
    b_obs = obs.reshape((-1,) + envs.single_observation_space.shape)
    if iteration == 1:
        t_obs = b_obs.clone()
    else:
        t_obs = torch.vstack((t_obs,b_obs))
    b_actions = actions.reshape((-1,) + envs.single_action_space.shape)
    b_returns = returns.reshape(-1)

    # Organize the input data with the collected frequency samples:

    state_repeated = input_data.reshape((-1,) + envs.single_observation_space.shape)
    omega_repeated = omega.repeat(b_obs.shape[0]).to(device)
    state_omega = torch.cat((state_repeated, omega_repeated.unsqueeze(1)), dim=1)

    # Organize the target data with the collected frequency samples:

    ecfSamples = output_data.reshape(-1)
    args.data_size_critic = state_omega.shape[0]
    num_batches = args.data_size_critic//args.batch_size 

    lossEpoch = np.zeros(args.epochs)
    b_inds = np.arange(args.data_size_critic)

    print(f'Size of critic training set is {len(b_inds)}')

    for epoch in range(args.epochs):
        np.random.shuffle(b_inds)
    
        frac = 1.0 - epoch/ args.epochs
        lrnow = frac * args.learning_rate
        model_optimizer.param_groups[0]["lr"] = lrnow

        for i in range(num_batches):

            start_idx = i * args.batch_size
            end_idx = start_idx + args.batch_size

            mb_inds = b_inds[start_idx:end_idx]
            
            inputs = state_omega[mb_inds]
            targets = ecfSamples[mb_inds]

            model_optimizer.zero_grad()  # Zero the gradients for the model parameters

            outputs = cf_network(inputs)

            loss_cfNN = torch.mean(torch.abs(outputs - targets)**2) 
            loss_cfNN.backward()
            model_optimizer.step()
    
            lossEpoch[epoch]+=loss_cfNN.item()

        print(f'Epoch {epoch+1}, Loss: {lossEpoch[epoch]}')
    
    model_copies.append(copy.deepcopy(cf_network.state_dict()))
    print(f'Iteration {iteration}')

In [ ]:
permuted_indices = torch.randperm(t_obs.size(0))
t_obs = t_obs[permuted_indices]
print(f'Size of observation set is {t_obs.shape[0]}')

In [ ]:
plt.hist(omega, bins=20, edgecolor='black')

In [ ]:
t_index = 2

In [ ]:
env = gym.make('CartPole-v1')
st_state = t_obs[t_index,:].numpy()
num_st = 100
r = np.zeros(num_st)
for i in range(num_st):
    _,_ = env.reset()
    episode_return = 0
    env.state = st_state
    
    state = st_state
    done = False
    truncated = False
    df = 1
    while not done and not truncated:
        action = agent.get_action(torch.Tensor(st_state)).numpy()
        state, reward, done, truncated, _ = env.step(action)
        episode_return += df*reward
        df = df*args.gamma
    r[i] = episode_return
print(np.mean(r))

In [ ]:
plt.hist(r, bins=20, edgecolor='black')

In [ ]:
def ecf(samples,omega):
    expo_samples = torch.exp(1j*omega*samples)  

    return torch.mean(expo_samples, dim=0)

vectorized_ecf = torch.vmap(ecf,in_dims=(None,0),chunk_size=100)
freq_values = torch.linspace(-2, 2, 1000)
ecfSamp = vectorized_ecf(torch.Tensor(r)*0.25, freq_values)

In [ ]:
cf_network.eval()
state = (t_obs[t_index,:].repeat(1000,1))
evalPlot = torch.cat((state,freq_values.unsqueeze(1)),dim=1)
cfNN = cf_network(evalPlot).detach()

plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
# plt.plot(freq_values.cpu(), ecf_eval.real.cpu(), label='Real Part (ECF)')
plt.plot(freq_values, cfNN.real.cpu(), label='Real Part (NN)')
plt.plot(freq_values, ecfSamp.real, label='Real Part (emp)')
plt.xlabel('Frequency')
plt.ylabel('Real Part')
plt.title(f'Real Component')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(freq_values.cpu(), cfNN.imag.cpu(), label='Imaginary Part (NN)')
plt.plot(freq_values.cpu(), ecfSamp.imag, label='Imaginary Part (emp)')
plt.xlabel('Frequency')
plt.ylabel('Imaginary Part')
plt.title(f'Imaginary Component')
plt.legend()
plt.suptitle(f'Plot for state {state[0].numpy()} at time index {t_index}')
plt.savefig(f't{t_index}'+f'_b{args.batch_size}'+'.png')

In [ ]:
torch.save({f'model_{i}': model for i, model in enumerate(model_copies)}, 'models_dict_batchs_'+f'{args.batch_size}.pt')

In [ ]:
models_dict = torch.load('models_dict_batchs_'+f'{args.batch_size}.pt')
cf_network.load_state_dict(models_dict[f'model_{args.num_iterations-1}'])